In [ ]:
%matplotlib inline
from PIL import Image
import glob
import pandas as pd
import yaml
import numpy as np
import cv2
import time
from skimage import exposure
from skimage import feature
import matplotlib.pyplot as plt

In [ ]:
import yaml
skip_lines = 3
def read_yaml_depth(file):
    file_yaml = file.replace('bmp', 'yaml')
    print(file_yaml)
    with open(file_yaml) as infile:
        for i in range(skip_lines):
            _ = infile.readline()
        data = yaml.load(infile)
    return np.reshape(data['data'], (data['rows'], data['cols']))

In [ ]:
show_image = True

def show_hog_img(img, hog_img, ero_img, ero_hog_img, dil_img, dil_hog_img, opening, closing, 
                 gradient, tophat,laplacian,sobelx,sobely):#,edges):

    for i in range(4):
        hog_img[i] = exposure.rescale_intensity(hog_img[i], out_range=(0, 255))
        hog_img[i] = hog_img[i].astype("uint8")
        
        ero_hog_img[i] = exposure.rescale_intensity(ero_hog_img[i], out_range=(0, 255))
        ero_hog_img[i] = ero_hog_img[i].astype("uint8")
        
        dil_hog_img[i] = exposure.rescale_intensity(dil_hog_img[i], out_range=(0, 255))
        dil_hog_img[i] = dil_hog_img[i].astype("uint8")
    
    img_concat = np.concatenate((img[0], img[1],img[2],img[3]), axis = 1)
    img_hog_concat = np.concatenate((hog_img[0], hog_img[1],hog_img[2],hog_img[3]), axis = 1)
    img_erosion_concat = np.concatenate((ero_img[0], ero_img[1],ero_img[2],ero_img[3]), axis = 1)
    img_erosion_hog_concat = np.concatenate((ero_hog_img[0], ero_hog_img[1],ero_hog_img[2],ero_hog_img[3]), axis = 1)
    img_dilation_concat = np.concatenate((dil_img[0], dil_img[1],dil_img[2],dil_img[3]), axis = 1)
    opening_concat = np.concatenate((opening[0], opening[1],opening[2],opening[3]), axis = 1)
    closing_concat = np.concatenate((closing[0], closing[1],closing[2],closing[3]), axis = 1)
    gradient_concat = np.concatenate((gradient[0], gradient[1],gradient[2],gradient[3]), axis = 1)
    tophat_concat = np.concatenate((tophat[0], tophat[1],tophat[2],tophat[3]), axis = 1)
    laplacian_concat = np.concatenate((laplacian[0], laplacian[1],laplacian[2],laplacian[3]), axis = 1)
    sobelx_concat = np.concatenate((sobelx[0], sobelx[1],sobelx[2],sobelx[3]), axis = 1)
    sobely_concat = np.concatenate((sobely[0], sobely[1],sobely[2],sobely[3]), axis = 1)
    #edges_concat = np.concatenate((edges[0], edges[1],edges[2],edges[3]), axis = 1)
    
    img_concat = np.concatenate((img[0], hog_img[0],ero_img[0],ero_hog_img[0], dil_img[0], opening[0],closing[0], gradient[0], tophat[0], laplacian[0], sobelx[0]), axis = 1)#, edges[0] ), axis = 1)
    cv2.imshow("Depth", img_concat)
    
    img_concat = np.concatenate((img[1], hog_img[1],ero_img[1],ero_hog_img[1], dil_img[1], opening[1],closing[1], gradient[1]), axis = 1)
    cv2.imshow("IR1", img_concat)
    img_concat = np.concatenate((tophat[1], laplacian[1], sobelx[1]), axis = 1)#, edges[1] ), axis = 1)
    cv2.imshow("IR1-2", img_concat)
    
    img_concat = np.concatenate((img[2], hog_img[2],ero_img[2],ero_hog_img[2], dil_img[2], opening[2],closing[2], gradient[2]), axis = 1)
    cv2.imshow("IR2", img_concat)
    img_concat = np.concatenate((tophat[2], laplacian[2], sobelx[2]), axis = 1)#, edges[2] ), axis = 1)
    cv2.imshow("IR2-2", img_concat)
    
    img_concat = np.concatenate((img[3], hog_img[3],ero_img[3],ero_hog_img[3], dil_img[3], opening[3],closing[3], gradient[3]), axis = 1)
    cv2.imshow("RGB", img_concat)
    img_concat = np.concatenate((tophat[3], laplacian[3], sobelx[3]), axis = 1)#, edges[3] ), axis = 1)
    cv2.imshow("RGB-2", img_concat)
    
    #img_total = np.concatenate((img_concat, 
                                #img_hog_concat, 
                                #img_erosion_concat, #img_erosion_hog_concat, 
                                #img_dilation_concat,#img_dilation_hog_concat
                                #opening_concat,
                                #closing_concat,
                                #gradient_concat,
                                #tophat_concat,
                                #laplacian_concat,
                                #sobelx_concat,
                                #sobely_concat, 
                                #edges_concat,
    #                           ), axis = 0)

    #cv2.imshow("HOG Image", img_total)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    #cv2.imwrite("teste.bmp", img_total)

In [ ]:
def calculate_hog(row):
    tic = time.clock()
    kernel = np.ones((3,3), np.uint8)
    
    # improvement agrupar
    #dep = cv2.imread(row['depfile_path'],0)
    dep = read_yaml_depth(row['depfile_path'])
    dep = dep[row['depxs']:row['depxe'],row['depys']:row['depye']]
    dep = (dep - dep.mean()) / dep.std()
    print(dep.shape, dep.mean(), dep.std())
    
    (H_dep, dep_hog) = feature.hog(dep, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    #print(dep.shape, dep_hog.shape, H_dep.shape)
    
    #dep.convertTo(dep, CV_8U);
    img_erosion_dep = cv2.erode(dep, kernel, iterations=1)
    (H_dep, dep_hog_ero) = feature.hog(img_erosion_dep, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    img_dilation_dep = cv2.dilate(dep, kernel, iterations=1)
    (H_dep, dep_hog_dil) = feature.hog(img_dilation_dep, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    opening_dep = cv2.morphologyEx(dep, cv2.MORPH_OPEN, kernel)
    closing_dep = cv2.morphologyEx(dep, cv2.MORPH_CLOSE, kernel)
    gradient_dep = cv2.morphologyEx(dep, cv2.MORPH_GRADIENT, kernel)
    tophat_dep = cv2.morphologyEx(dep, cv2.MORPH_TOPHAT, kernel)
    laplacian_dep = cv2.Laplacian(dep,cv2.CV_64F)
    sobelx_dep = cv2.Sobel(dep,cv2.CV_64F,1,0,ksize=5)
    sobely_dep = cv2.Sobel(dep,cv2.CV_64F,0,1,ksize=5)
    edges_dep = np.uint8(dep)
    edges_dep = cv2.Canny(edges_dep,100,200)
    
    ir1 = cv2.imread(row['ir1file_path'],0)
    ir1 = ir1[row['ir1xs']:row['ir1xe'],row['ir1ys']:row['ir1ye']]
    ir1 = (ir1 - ir1.mean()) / ir1.std()
    print(ir1.mean(), ir1.std())
    
    (H_ir1, ir1_hog) = feature.hog(ir1, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    #print(ir1.shape, ir1_hog.shape, H_ir1.shape)
    img_erosion_ir1 = cv2.erode(ir1, kernel, iterations=1)
    (H_dep, ir1_hog_ero) = feature.hog(img_erosion_ir1, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    img_dilation_ir1 = cv2.dilate(ir1, kernel, iterations=1)
    (H_dep, ir1_hog_dil) = feature.hog(img_dilation_ir1, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    opening_ir1 = cv2.morphologyEx(ir1, cv2.MORPH_OPEN, kernel)
    closing_ir1 = cv2.morphologyEx(ir1, cv2.MORPH_CLOSE, kernel)
    gradient_ir1 = cv2.morphologyEx(ir1, cv2.MORPH_GRADIENT, kernel)
    tophat_ir1 = cv2.morphologyEx(ir1, cv2.MORPH_TOPHAT, kernel)
    laplacian_ir1 = cv2.Laplacian(ir1,cv2.CV_64F)
    sobelx_ir1 = cv2.Sobel(ir1,cv2.CV_64F,1,0,ksize=5)
    sobely_ir1 = cv2.Sobel(ir1,cv2.CV_64F,0,1,ksize=5)
    #edges_ir1 = cv2.Canny(ir1,100,200)
    
    ir2 = cv2.imread(row['ir2file_path'],0)
    ir2 = ir2[row['ir2xs']:row['ir2xe'],row['ir2ys']:row['ir2ye']]
    #ir2 = (ir2 - ir2.mean()) / ir2.std()
    print(ir2.mean(), ir2.std())
    
    (H_ir2, ir2_hog) = feature.hog(ir2, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    #print(ir2.shape, ir2_hog.shape, H_ir2.shape)
    img_erosion_ir2 = cv2.erode(ir2, kernel, iterations=1)
    (H_dep, ir2_hog_ero) = feature.hog(img_erosion_ir2, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    img_dilation_ir2 = cv2.dilate(ir2, kernel, iterations=1)
    (H_dep, ir2_hog_dil) = feature.hog(img_dilation_ir2, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    opening_ir2 = cv2.morphologyEx(ir2, cv2.MORPH_OPEN, kernel)
    closing_ir2 = cv2.morphologyEx(ir2, cv2.MORPH_CLOSE, kernel)
    gradient_ir2 = cv2.morphologyEx(ir2, cv2.MORPH_GRADIENT, kernel)
    tophat_ir2 = cv2.morphologyEx(ir2, cv2.MORPH_TOPHAT, kernel)
    laplacian_ir2 = cv2.Laplacian(ir2,cv2.CV_64F)
    sobelx_ir2 = cv2.Sobel(ir2,cv2.CV_64F,1,0,ksize=5)
    sobely_ir2 = cv2.Sobel(ir2,cv2.CV_64F,0,1,ksize=5)
    #edges_ir2 = cv2.Canny(ir2,100,200)
    
    rgb = cv2.imread(row['rgbfile_path'],0)
    rgb = rgb[row['rgbxs']:row['rgbxe'],row['rgbys']:row['rgbye']]
    #rgb = (rgb - rgb.mean()) / rgb.std()
    print(rgb.shape, rgb.mean(), rgb.std())
    
    (H_rgb, rgb_hog) = feature.hog(rgb, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    #print(rgb.shape, rgb_hog.shape, H_rgb.shape)
    img_erosion_rgb = cv2.erode(rgb, kernel, iterations=1)
    (H_dep, rgb_hog_ero) = feature.hog(img_erosion_rgb, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    img_dilation_rgb = cv2.dilate(rgb, kernel, iterations=1)
    (H_dep, rgb_hog_dil) = feature.hog(img_dilation_rgb, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    opening_rgb = cv2.morphologyEx(rgb, cv2.MORPH_OPEN, kernel)
    closing_rgb = cv2.morphologyEx(rgb, cv2.MORPH_CLOSE, kernel)
    gradient_rgb = cv2.morphologyEx(rgb, cv2.MORPH_GRADIENT, kernel)
    tophat_rgb = cv2.morphologyEx(rgb, cv2.MORPH_TOPHAT, kernel)
    laplacian_rgb = cv2.Laplacian(rgb,cv2.CV_64F)
    sobelx_rgb = cv2.Sobel(rgb,cv2.CV_64F,1,0,ksize=5)
    sobely_rgb = cv2.Sobel(rgb,cv2.CV_64F,0,1,ksize=5)
    #edges_rgb = cv2.Canny(rgb,100,200)
    
    if show_image == True:
        show_hog_img([dep,ir1,ir2,rgb], 
                     [dep_hog, ir1_hog, ir2_hog, rgb_hog], 
                     [img_erosion_dep, img_erosion_ir1, img_erosion_ir2, img_erosion_rgb], 
                     [dep_hog_ero, ir1_hog_ero, ir2_hog_ero, rgb_hog_ero], 
                     [img_dilation_dep, img_dilation_ir1, img_dilation_ir2, img_dilation_rgb], 
                     [dep_hog_dil, ir1_hog_dil, ir2_hog_dil, rgb_hog_dil],
                     [opening_dep, opening_ir1, opening_ir2, opening_rgb],
                     [closing_dep, closing_ir1, closing_ir2, closing_rgb],
                     [gradient_dep, gradient_ir1, gradient_ir2, gradient_rgb],
                     [tophat_dep, tophat_ir1, tophat_ir2, tophat_rgb],
                     [laplacian_dep,laplacian_ir1,laplacian_ir2,laplacian_rgb],
                     [sobelx_dep,sobelx_ir1,sobelx_ir2,sobelx_rgb],
                     [sobely_dep,sobely_ir1,sobely_ir2,sobely_rgb],
                     #[edges_dep,edges_ir1,edges_ir2,edges_rgb]
                     #[0,0,0,0]
                    )
        
    print(ds.shape, row['depfile_path'], (time.clock() - tic))

In [ ]:
prefix = 'C:\\Users\\rafae\\Desktop\\Coleta\\faces\\training_p012_10.csv'

In [6]:
ds = pd.read_csv(prefix, header=0)

In [7]:
ds.head()

class person                             depfile_path  depxs  depys  depxe  \
0     10   p012   E:\MestradoData\p012\12_10_depth_1.bmp     86    153    190   
1     10   p012  E:\MestradoData\p012\12_10_depth_11.bmp     84    152    188   
2     10   p012  E:\MestradoData\p012\12_10_depth_21.bmp     83    152    187   
3     10   p012  E:\MestradoData\p012\12_10_depth_31.bmp     83    152    187   
4     10   p012  E:\MestradoData\p012\12_10_depth_41.bmp     79    152    183   

   depye                           ir1file_path  ir1xs  ir1ys  ...    \
0    241   E:\MestradoData\p012\12_10_ir1_1.bmp     86    153  ...     
1    240  E:\MestradoData\p012\12_10_ir1_11.bmp     84    152  ...     
2    240  E:\MestradoData\p012\12_10_ir1_21.bmp     83    152  ...     
3    240  E:\MestradoData\p012\12_10_ir1_31.bmp     83    152  ...     
4    240  E:\MestradoData\p012\12_10_ir1_41.bmp     79    152  ...     

                            ir2file_path  ir2xs ir2ys  ir2xe  ir2ye  \
0   E:\MestradoData\p012\12_10_ir2_1.bmp     86   123    190    211   
1  E:\MestradoData\p012\12_10_ir2_11.bmp     84   122    188    210   
2  E:\MestradoData\p012\12_10_ir2_21.bmp     83   122    187    210   
3  E:\MestradoData\p012\12_10_ir2_31.bmp     83   122    187    210   
4  E:\MestradoData\p012\12_10_ir2_41.bmp     79   122    183    210   

                            rgbfile_path  rgbxs rgbys  rgbxe  rgbye  
0   E:\MestradoData\p012\12_10_rgb_1.bmp     81   128    185    216  
1  E:\MestradoData\p012\12_10_rgb_11.bmp     79   127    183    215  
2  E:\MestradoData\p012\12_10_rgb_21.bmp     78   127    182    215  
3  E:\MestradoData\p012\12_10_rgb_31.bmp     78   127    182    215  
4  E:\MestradoData\p012\12_10_rgb_41.bmp     74   127    178    215  

[5 rows x 22 columns]

In [8]:
calculate_hog(ds.iloc[0])

E:\MestradoData\p012\12_10_depth_1.yaml
(104, 88) -4.96883032e-17 1.0


C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:144: RuntimeWarning: invalid value encountered in sqrt
  image = np.sqrt(image)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:190: RuntimeWarning: invalid value encountered in remainder
  orientations, orientation_histogram)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:17: RuntimeWarning: invalid value encountered in minimum
  out = np.minimum(out, 0.2)


-6.2110379e-17 1.0
111.348011364 64.2536176431
(104, 88) 120.528190559 84.4372198243
(15, 22) E:\MestradoData\p012\12_10_depth_1.bmp 2717.449480691358
